# Rolling Stone Album Rankings Analysis
### Predicting Album Longevity using Machine Learning and Survival Analysis

**Author**: Tamas Nemeth  
**Data Source**: TidyTuesday Rolling Stone Top 500 Albums  
**Tools**: Python, scikit-learn, OpenAI API, Lifelines, Pandas

---

## Project Overview

This analysis investigates what characteristics make an album endure on the Rolling Stone Top 500 list over time. Using the Rolling Stone rankings from 2003, 2012, and 2020, I built predictive models to identify factors that contribute to an album's lasting success.

**Key Techniques Used**:
- **LLM-Enhanced Data Imputation**: Using GPT-4 to fill missing genre data
- **Survival Analysis**: Kaplan-Meier estimators for understanding album longevity
- **Advanced Feature Engineering**: Custom binary indicators and systematic feature selection
- **Robust Cross-Validation**: 20-repeat stratified k-fold validation

**Main Findings**:
- 92% of albums from 2003 remained on the list in 2012
- 64% of 2003 albums were still ranked in 2020
- Album type (Studio vs Live) emerged as the strongest predictor
- Genre-specific patterns (Funk/Disco) showed significant predictive power

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from openai import OpenAI
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report, confusion_matrix
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2024/2024-05-07/rolling_stone.csv")
print("Info about the dataset:")
print(data.info())
print("-"*101)
print("Missing values in the dataset:")
print(data.isnull().sum())


Info about the dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   sort_name                 691 non-null    object 
 1   clean_name                691 non-null    object 
 2   album                     691 non-null    object 
 3   rank_2003                 500 non-null    float64
 4   rank_2012                 500 non-null    float64
 5   rank_2020                 500 non-null    float64
 6   differential              691 non-null    int64  
 7   release_year              691 non-null    int64  
 8   genre                     527 non-null    object 
 9   type                      691 non-null    object 
 10  weeks_on_billboard        572 non-null    float64
 11  peak_billboard_position   691 non-null    int64  
 12  spotify_popularity        654 non-null    float64
 13  spotify_url               655 non-null   



`sort_name`		                Name used for sorting.

`clean_name`	                Clean name.

`album`	    	                Album name.

`rank_2003`	                    Rank in 2003. NA if album not released yet or not in top 500.

`rank_2012`		                Rank in 2012. NA if album not released yet or not in top 500.

`rank_2020`		                Rank in 2020. NA if not in top 500.

`differential`	                2020-2003 Differential. Negative value if it went down in the chart. Positive value if it went up.

`release_year`	                Release Year.

`genre`	    	                Album Genre.

`type`	                        Album Type.

`weeks_on_billboard`		    Weeks on Billboard.

`peak_billboard_position`		Peak Billboard Position.

`spotify_popularity`	    	Spotify Popularity. NA if not on Spotify.

`spotify_url`	                Spotify URL. NA if not on Spotify.

`artist_member_count`		    Number of artists in the group.

`artist_gender`		            Gender of the artist(s). Male/Female if it's a mixed-gender group.

`artist_birth_year_sum`	    	Sum of the artists birth year. e.g. for a 2 member group, with one person born 1945 and another 1950, the value is 3895.

`debut_album_release_year`		Debut Album Release Year.

`ave_age_at_top_500`		    Average age at top 500 Album.

`years_between`		            Years Between Debut and Top 500 Album.

`album_id`	        	        Album ID. NOS at the beginning of the ID if not on Spotify.


In [3]:
filtered_data = data[data["rank_2003"].notna()].copy()
display(filtered_data)

,sort_name,clean_name,album,rank_2003,rank_2012,rank_2020,differential,release_year,genre,type,...,peak_billboard_position,spotify_popularity,spotify_url,artist_member_count,artist_gender,artist_birth_year_sum,debut_album_release_year,ave_age_at_top_500,years_between,album_id
0,"Sinatra, Frank",Frank Sinatra,In the Wee Small Hours,100.0,101.0,282.0,-182,1955,Big Band/Jazz,Studio,...,2,48.0,spotify:album:3GmwKB1tgPZgXeRJZSm9WX,1.0,Male,1915.0,1946.0,40.0,9.0,3GmwKB1tgPZgXeRJZSm9WX
1,"Diddley, Bo",Bo Diddley,Bo Diddley / Go Bo Diddley,214.0,216.0,455.0,-241,1955,Rock n' Roll/Rhythm & Blues,Studio,...,201,50.0,spotify:album:1cbtDEwxCjMhglb49OgNBR,1.0,Male,1928.0,1955.0,27.0,0.0,1cbtDEwxCjMhglb49OgNBR
2,"Presley, Elvis",Elvis Presley,Elvis Presley,55.0,56.0,332.0,-277,1956,Rock n' Roll/Rhythm & Blues,Studio,...,1,58.0,spotify:album:7GXP5OhYyPVLmcVfO9Iqin,1.0,Male,1935.0,1956.0,21.0,0.0,7GXP5OhYyPVLmcVfO9Iqin
3,"Sinatra, Frank",Frank Sinatra,Songs for Swingin' Lovers!,306.0,308.0,NaN,-195,1956,Big Band/Jazz,Studio,...,2,62.0,spotify:album:4kca7vXd1Wo5GE2DMafvMc,1.0,Male,1915.0,1946.0,41.0,10.0,4kca7vXd1Wo5GE2DMafvMc
4,Little Richard,Little Richard,Here's Little Richard,50.0,50.0,227.0,-177,1957,NaN,Studio,...,13,64.0,spotify:album:18tV6PLXYvVjsdOVk0S7M8,1.0,Male,1932.0,1957.0,25.0,0.0,18tV6PLXYvVjsdOVk0S7M8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,Eminem,Eminem,The Eminem Show,317.0,NaN,NaN,-184,2002,Hip-Hop/Rap,Studio,...,1,87.0,spotify:album:2cWBwpqMsDJC1ZUwz813lo,1.0,Male,1972.0,1996.0,30.0,6.0,2cWBwpqMsDJC1ZUwz813lo
685,"Wolf, Peter",Peter Wolf,Sleepless,432.0,427.0,NaN,-69,2002,NaN,Studio,...,201,30.0,spotify:album:6XZwHDglhO4A0C5Pa4xO4Q,1.0,Male,1946.0,1984.0,56.0,18.0,6XZwHDglhO4A0C5Pa4xO4Q
686,Beck,Beck,Sea Change,440.0,436.0,NaN,-61,2002,Indie/Alternative Rock,Studio,...,8,51.0,spotify:album:5ieP11rJQvuYz0Ov3k03cy,1.0,Male,1970.0,1993.0,32.0,9.0,5ieP11rJQvuYz0Ov3k03cy
687,"Cooke, Sam",Sam Cooke,Portrait of a Legend 1951-1964,106.0,107.0,307.0,-201,2003,Soul/Gospel/R&B,Greatest Hits,...,135,55.0,spotify:album:4jiO2jRz7g50ESvYYKsKwZ,1.0,Male,1931.0,1958.0,72.0,45.0,4jiO2jRz7g50ESvYYKsKwZ


I removed every album that did not exist in 2003.  
Next I create 3 binary variables:  
`billboard_2003`  
`billboard_2012`  
`billboard_2020`  
*With the following values*  
`1` Was on the Billboard  
`0` Was not on Billboard

In [63]:
filtered_data["billboard_2003"] = 1 # since every album was on the billboard in 2003  
filtered_data["billboard_2012"] = filtered_data["rank_2012"].notna().astype(int)
filtered_data["billboard_2020"] = filtered_data["rank_2020"].notna().astype(int)

For the first analysis I am curious what makes an album a good album, so first I am creating a model from 2003 to 2012, and then based on the results predicting 2020.

Since genre is a very important predictor, and I have missing values, I used GPT to fill in their data.

In [5]:
data_for_llm = filtered_data.copy()


In [6]:
non_missing_data = data_for_llm[data_for_llm["genre"].notna()]
non_missing_data["genre"].value_counts()

genre
Punk/Post-Punk/New Wave/Power Pop      71
Blues/Blues Rock                       60
Soul/Gospel/R&B                        45
Country/Folk/Country Rock/Folk Rock    36
Indie/Alternative Rock                 35
Singer-Songwriter/Heartland Rock       26
Hip-Hop/Rap                            26
Hard Rock/Metal                        25
Funk/Disco                             15
Big Band/Jazz                          10
Rock n' Roll/Rhythm & Blues            10
Reggae                                  7
Electronic                              7
Blues/Blues ROck                        1
Latin                                   1
Name: count, dtype: int64

In [7]:
# Find rows with missing values
missing_genre_mask = data_for_llm["genre"].isna()

# Create distinct dataset with only missing values
missing_data = data_for_llm[missing_genre_mask].copy()

In [ ]:
display(missing_data)

In [9]:
load_dotenv()

# Get API key
api_key = os.getenv("OPENAI_API_KEY")

# check
if not api_key:
    raise ValueError("API_KEY not found in environment variables")



In [10]:
client = OpenAI()

total_rows = len(missing_data)
counter = 0

for index, row in missing_data.iterrows():
    counter += 1
    print(f"Processing {counter}/{total_rows}: {row['album']} by {row['clean_name']}")
    
    response = client.chat.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        messages=[
            {"role": "system", "content": "You are a music expert. Provide concise and accurate genre classifications for albums based on the given options."},
            {"role": "user", "content": f"""Based on the album '{row['album']}' and the artist '{row['clean_name']}',
                        give me the genre of the album if it is missing.
                        The genre can be one of the following: {list(genre)}.
                        If you don't know the genre, leave it blank.
                        If the genre is not in the list, write a new one. """}
        ],
        max_completion_tokens=20, 
        temperature=0  
    )
    
    response_text = response.choices[0].message.content.strip()
    print(f"Response: {response_text}")
    missing_data.loc[index, "genre"] = response_text

display(missing_data)

Processing 1/125: Here's Little Richard by Little Richard
Response: Rock n' Roll/Rhythm & Blues
Processing 2/125: The Genius of Ray Charles by Ray Charles
Response: Soul/Gospel/R&B
Processing 3/125: Please Please Me by The Beatles
Response: Rock n' Roll/Rhythm & Blues
Processing 4/125: With the Beatles by The Beatles
Response: Rock n' Roll/Rhythm & Blues
Processing 5/125: Rubber Soul by The Beatles
Response: Rock n' Roll/Rhythm & Blues
Processing 6/125: Help! by The Beatles
Response: Rock n' Roll/Rhythm & Blues
Processing 7/125: Mr. Tambourine Man by The Byrds
Response: Country/Folk/Country Rock/Folk Rock
Processing 8/125: The Beach Boys Today! by The Beach Boys
Response: Rock n' Roll/Rhythm & Blues
Processing 9/125: Pet Sounds by The Beach Boys
Response: Rock n' Roll/Rhythm & Blues
Processing 10/125: Revolver by The Beatles
Response: Rock n' Roll/Rhythm & Blues
Processing 11/125: Freak Out! by The Mothers of Invention
Response: Rock n' Roll/Rhythm & Blues
Processing 12/125: If You Can

,sort_name,clean_name,album,rank_2003,rank_2012,rank_2020,differential,release_year,genre,type,...,artist_member_count,artist_gender,artist_birth_year_sum,debut_album_release_year,ave_age_at_top_500,years_between,album_id,bilboard_2003,bilboard_2012,bilboard_2020
4,Little Richard,Little Richard,Here's Little Richard,50.0,50.0,227.0,-177,1957,Rock n' Roll/Rhythm & Blues,Studio,...,1.0,Male,1932.0,1957.0,25.00,0.0,18tV6PLXYvVjsdOVk0S7M8,1,1,1
18,"Charles, Ray",Ray Charles,The Genius of Ray Charles,263.0,265.0,NaN,-238,1959,Soul/Gospel/R&B,Studio,...,1.0,Male,1930.0,1949.0,29.00,10.0,4GFWnwli2cVOBp2G1zqhV1,1,1,0
43,Beatles,The Beatles,Please Please Me,39.0,39.0,NaN,-462,1963,Rock n' Roll/Rhythm & Blues,Studio,...,4.0,Male,7765.0,1963.0,21.75,0.0,3KzAvEXcqJKBF97HrXwlgf,1,1,0
46,Beatles,The Beatles,With the Beatles,420.0,NaN,NaN,-81,1963,Rock n' Roll/Rhythm & Blues,Studio,...,4.0,Male,7765.0,1963.0,21.75,0.0,1aYdiJk6XKeHWGO3FzHHTr,1,0,0
81,Beatles,The Beatles,Rubber Soul,5.0,5.0,35.0,-30,1965,Rock n' Roll/Rhythm & Blues,Studio,...,4.0,Male,7765.0,1963.0,23.75,2.0,50o7kf2wLwVmOTVYJOTplm,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,"Diamond, Neil",Neil Diamond,The Neil Diamond Collection,222.0,224.0,NaN,-279,1999,Singer-Songwriter/Heartland Rock,Greatest Hits,...,1.0,Male,1951.0,1966.0,48.00,33.0,NOS129,1,1,0
651,"Waits, Tom",Tom Waits,Mule Variations,416.0,416.0,NaN,-85,1999,Blues/Blues Rock,Studio,...,1.0,Male,1949.0,1973.0,50.00,26.0,7cAcex6xw4fP67ltgn1gm3,1,1,0
665,ABBA,ABBA,The Definitive Collection,180.0,179.0,303.0,-123,2001,Funk/Disco,Studio,...,4.0,Male/Female,7786.0,1973.0,54.50,28.0,34lTW6LuORpuIiYfqsetuq,1,1,1
671,"Ronstadt, Linda",Linda Ronstadt,The Very Best of Linda Ronstadt,324.0,164.0,NaN,-177,2002,Country/Folk/Country Rock/Folk Rock,Greatest Hits,...,1.0,Female,1946.0,1969.0,56.00,33.0,NOS134,1,1,0


In [11]:
# Backup the data.
missing_data.to_excel("missing_genre.xlsx", index=False)

Since the model is a small model, I check it's accuracy with an LLM-as-a-Judge

In [12]:
missing_data_to_judge = missing_data.copy()
random_data = missing_data_to_judge.sample(10, random_state=123)

total_rows = len(random_data)
counter = 0



for index, row in random_data.iterrows():
    counter += 1
    print(f"Processing {counter}/{total_rows}: {row['album']} by {row['clean_name']}")
    
    response = client.chat.completions.create(
        model="gpt-4.1-2025-04-14",
        messages=[
            {"role": "system", "content": "You are a judge, judging another models response. Respond only with a number 1, 2. or 3."},
            {"role": "user", "content": f"""A smaller model was given a task to classify the genre of albums, 
                        based on the album '{row['album']}' and the artist '{row['clean_name']}'.
                        The genre can be one of the following: {list(genre)}.
                        If it did not know the genre, it was asked to leave it blank.
                        If the genre was not in the list, it wrote a new one. 
                        Your job is to judge the response of the model '{row['genre']}'.
                        Rank the response on a scale from 1 to 3, where:
                        1 - The response is completely wrong or irrelevant.
                        2 - The response is partially correct.
                        3 - The response is 100% correct.
                        
                        Format your response as: [Score]: [Brief reasoning]
                        If the score is 3, instead of reasoning write:
                        3: 3
                        If it's 1 or 2, write a brief reasoning why the score is not 3."""}
        ],
        max_completion_tokens=100, 
        temperature=0  
    )
    
    response_text = response.choices[0].message.content.strip()
    print(f"Response: {response_text}")
    
    # Parse score and reasoning
    parts = response_text.split(':', 1)
    score = parts[0].strip()
    reasoning = parts[1].strip() 
    
    random_data.loc[index, 'correctness_score'] = score
    random_data.loc[index, 'reasoning'] = reasoning



Processing 1/10: Rubber Soul by The Beatles
Response: 2: While "Rubber Soul" by The Beatles contains elements of "Rock n' Roll/Rhythm & Blues," it is more accurately classified as "Country/Folk/Country Rock/Folk Rock" or "Singer-Songwriter/Heartland Rock" due to its strong folk and singer-songwriter influences. The chosen genre is partially correct but not the best fit.
Processing 2/10: The Sun Records Collection by Various Artists
Response: 3: 3
Processing 3/10: The Wall by Pink Floyd
Response: 2: "The Wall" by Pink Floyd is best classified as "Rock" or "Progressive Rock," but "Rock n' Roll/Rhythm & Blues" is only partially correct since it does not specifically capture the progressive/art rock style of the album.
Processing 4/10: Another Green World by Brian Eno
Response: 3: 3
Processing 5/10: 12 Songs by Randy Newman
Response: 3: 3
Processing 6/10: Child Is Father to the Man by Blood, Sweat & Tears
Response: 2: While "Child Is Father to the Man" by Blood, Sweat & Tears incorporates 

In [13]:
#Backup
random_data.to_excel("random_data.xlsx", index=False)

In [14]:
display(random_data[["correctness_score", "reasoning"]])

,correctness_score,reasoning
81,2,"While ""Rubber Soul"" by The Beatles contains el..."
621,3,3
460,2,"""The Wall"" by Pink Floyd is best classified as..."
355,3,3
212,3,3
158,2,"While ""Child Is Father to the Man"" by Blood, S..."
160,2,The album is more accurately classified as 'In...
590,3,3
506,3,3
177,3,3


Although 4 out of 10 were incorrect, it still got 2 out of 3 correct. Inspecting the reasoning for the score of 2, I decided to move on with the model, since it is just a nuanced difference.

In [15]:
missing_data["genre"].value_counts()

genre
Rock n' Roll/Rhythm & Blues            52
Singer-Songwriter/Heartland Rock       23
Country/Folk/Country Rock/Folk Rock    22
Soul/Gospel/R&B                         7
Funk/Disco                              6
Blues/Blues Rock                        5
Electronic                              5
Psych Rock                              1
Big Band/Jazz                           1
Punk/Post-Punk/New Wave/Power Pop       1
Hip-Hop/Rap                             1
Indie/Alternative Rock                  1
Name: count, dtype: int64

In [16]:
#rs_data = Rolling Stones data 
rs_data =pd.concat([missing_data, non_missing_data], ignore_index=True)
#Backup
rs_data.to_excel("rs_data.xlsx", index=False)

In [17]:
rs_data= pd.read_excel("rs_data.xlsx")

# Survival analysis of the albums

In [ ]:
# Create survival variables: duration and event
rs_data['duration'] = 0
rs_data['event'] = 0

# Create duration and event for each album
for index, row in rs_data.iterrows():
    if row['billboard_2012'] == 0:
        # Album dropped off between 2003-2012
        rs_data.loc[index, 'duration'] = 9   # 2012 - 2003
        rs_data.loc[index, 'event'] = 1      # Event occurred (dropped off)
    elif row['billboard_2020'] == 0:
        # Album stayed until 2012, but dropped off by 2020
        rs_data.loc[index, 'duration'] = 17  # 2020 - 2003
        rs_data.loc[index, 'event'] = 1      # Event occurred (dropped off)
    else:
        # Album still on Billboard in 2020
        rs_data.loc[index, 'duration'] = 17  # 2020 - 2003
        rs_data.loc[index, 'event'] = 0      # Censored (still alive)


print("Survival Variables Created:")
print("Duration distribution:")
print(rs_data['duration'].value_counts().sort_index())
print("\nEvent distribution:")
print(rs_data['event'].value_counts())
print(f"\nTotal albums: {len(rs_data)}")
print(f"Albums that dropped off (events): {rs_data['event'].sum()}")
print(f"Albums still on list (censored): {(rs_data['event'] == 0).sum()}")

In [ ]:
kmf = KaplanMeierFitter()
kmf.fit(rs_data['duration'], rs_data['event'])

# Plot overall survival curve
plt.figure(figsize=(12, 6))
kmf.plot_survival_function(label='All Albums')
plt.title('Album Survival on Rolling Stone Top 500 (2003-2020)')
plt.xlabel('Years since 2003')
plt.ylabel('Probability of Staying on Top 500')
plt.axvline(x=9, color='red', linestyle='--', alpha=0.7, label='2012 checkpoint')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Print survival statistics
print("Overall Survival Statistics:")
print(f"Survival probability at 9 years (2012): {kmf.survival_function_at_times(9).iloc[0]:.3f}")
print(f"Survival probability at 17 years (2020): {kmf.survival_function_at_times(17).iloc[0]:.3f}")

Analysis shows that 92% of the albums that were on the Billboard charts in 2003 were still present in 2012, and nearly 64% of 2003 albums were on the 2020 Billboard charts. But what makes for that success? In the following Logistic Regression models I try to answer this question.

I created a word count for album names, because I am curious if shorter names help albums perform better.

In [18]:
for index, rows in rs_data.iterrows():
    words= word_tokenize(rows["album"])
    word_count = len(words)
    rs_data.loc[index, "album_name_word_count"] = word_count




In [23]:
display(rs_data)

,sort_name,clean_name,album,rank_2003,rank_2012,rank_2020,differential,release_year,genre,type,...,debut_album_release_year,ave_age_at_top_500,years_between,album_id,bilboard_2003,bilboard_2012,bilboard_2020,album_name_word_count,2003_2012,remained_2012
0,Little Richard,Little Richard,Here's Little Richard,50,50.0,227.0,-177,1957,Rock n' Roll/Rhythm & Blues,Studio,...,1957.0,25.00,0.0,18tV6PLXYvVjsdOVk0S7M8,1,1,1,4.0,0,1
1,"Charles, Ray",Ray Charles,The Genius of Ray Charles,263,265.0,NaN,-238,1959,Soul/Gospel/R&B,Studio,...,1949.0,29.00,10.0,4GFWnwli2cVOBp2G1zqhV1,1,1,0,5.0,0,1
2,Beatles,The Beatles,Please Please Me,39,39.0,NaN,-462,1963,Rock n' Roll/Rhythm & Blues,Studio,...,1963.0,21.75,0.0,3KzAvEXcqJKBF97HrXwlgf,1,1,0,3.0,0,1
3,Beatles,The Beatles,With the Beatles,420,NaN,NaN,-81,1963,Rock n' Roll/Rhythm & Blues,Studio,...,1963.0,21.75,0.0,1aYdiJk6XKeHWGO3FzHHTr,1,0,0,3.0,1,0
4,Beatles,The Beatles,Rubber Soul,5,5.0,35.0,-30,1965,Rock n' Roll/Rhythm & Blues,Studio,...,1963.0,23.75,2.0,50o7kf2wLwVmOTVYJOTplm,1,1,1,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Coldplay,Coldplay,A Rush of Blood to the Head,473,466.0,324.0,149,2002,Indie/Alternative Rock,Studio,...,2000.0,24.50,2.0,0RHX9XECH8IVI3LNgWDpmQ,1,1,1,7.0,0,1
496,Eminem,Eminem,The Eminem Show,317,NaN,NaN,-184,2002,Hip-Hop/Rap,Studio,...,1996.0,30.00,6.0,2cWBwpqMsDJC1ZUwz813lo,1,0,0,3.0,1,0
497,Beck,Beck,Sea Change,440,436.0,NaN,-61,2002,Indie/Alternative Rock,Studio,...,1993.0,32.00,9.0,5ieP11rJQvuYz0Ov3k03cy,1,1,0,2.0,0,1
498,"Cooke, Sam",Sam Cooke,Portrait of a Legend 1951-1964,106,107.0,307.0,-201,2003,Soul/Gospel/R&B,Greatest Hits,...,1958.0,72.00,45.0,4jiO2jRz7g50ESvYYKsKwZ,1,1,1,5.0,0,1


In [24]:
rs_data["billboard_2012"].value_counts()

bilboard_2012
1    464
0     36
Name: count, dtype: int64

In [21]:
rs_data['type'] = rs_data['type'].replace('Soundtrack', 'Live')


In [25]:
def group_rare_categories(df, column, min_count=15):
    value_counts = df[column].value_counts()
    rare_categories = value_counts[value_counts < min_count].index
    df_copy = df.copy()
    df_copy[column] = df_copy[column].replace(rare_categories, 'Other')
    return df_copy 

rs_data_new = group_rare_categories(rs_data, 'genre', min_count=15)
ml_data = rs_data_new[["billboard_2012", "release_year", "type", "genre", "album_name_word_count", "artist_gender","debut_album_release_year", "years_between", "artist_member_count"]].dropna()
numeric_features = ["release_year", "album_name_word_count", "debut_album_release_year", "years_between", "artist_member_count"]
categorical_features = ["type","genre", "artist_gender"]
X= ml_data.drop(columns=["billboard_2012"])
y= ml_data["billboard_2012"]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=42
) # n_repeats=20, because I want as robust results as possible
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), categorical_features)
    ]
)

# Pipeline
pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=5000)
)

scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc")
print(scores)







[0.46171171 0.67027027 0.6826484  0.38812785 0.41552511 0.55630631
 0.46216216 0.44292237 0.60730594 0.47031963 0.70945946 0.41081081
 0.43607306 0.57305936 0.43835616 0.4954955  0.48378378 0.46347032
 0.39497717 0.63926941 0.38513514 0.58108108 0.52283105 0.65753425
 0.57305936 0.52477477 0.51891892 0.59817352 0.39269406 0.51369863
 0.61036036 0.50540541 0.5913242  0.43378995 0.79680365 0.48873874
 0.37837838 0.52283105 0.51826484 0.63242009 0.5990991  0.62972973
 0.39497717 0.64840183 0.46803653 0.45495495 0.65405405 0.52054795
 0.53196347 0.60159817 0.4481982  0.41891892 0.58447489 0.64840183
 0.46803653 0.6509009  0.64054054 0.34931507 0.43607306 0.51484018
 0.5472973  0.61351351 0.36986301 0.61643836 0.32876712 0.52027027
 0.52972973 0.59360731 0.36073059 0.56164384 0.74774775 0.38243243
 0.55707763 0.7260274  0.40639269 0.63513514 0.62162162 0.3196347
 0.46575342 0.6803653  0.31081081 0.66756757 0.59817352 0.57990868
 0.71689498 0.53828829 0.68648649 0.50456621 0.26141553 0.66666

In [26]:

from sklearn.metrics import roc_auc_score, classification_report

# Fit the pipeline before making predictions
pipeline.fit(X_train, y_train)

# Correct way to get ROC-AUC
y_pred_proba = pipeline.predict_proba(X_holdout)[:, 1]  # Probabilities for class 1
holdout_auc = roc_auc_score(y_holdout, y_pred_proba)
print(f"Holdout ROC-AUC: {holdout_auc:.3f}")

# For classification report
y_pred = pipeline.predict(X_holdout)  # Hard predictions
print(classification_report(y_holdout, y_pred))

Holdout ROC-AUC: 0.513
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.93      1.00      0.96        93

    accuracy                           0.93       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.86      0.93      0.90       100



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Well... The model is not the best... Let's do some feature engineering stuff. 

In [ ]:
# Your current baseline
baseline_score = 0.513

# Test removing each feature with full CV
current_features = ["release_year", "type", "genre", "album_name_word_count", 
                   "artist_gender", "debut_album_release_year", "years_between", 
                   "artist_member_count"]

print("Testing feature removal with cross-validation:")
print(f"Baseline (all features): {baseline_score:.3f}")
print("-" * 50)

feature_removal_results = {}

for feature_to_remove in current_features:
    print(f"Testing removal of: {feature_to_remove}")
    
    # Create feature lists without this feature
    remaining_features = [f for f in current_features if f != feature_to_remove]
    remaining_numeric = [f for f in remaining_features if f in numeric_features]
    remaining_categorical = [f for f in remaining_features if f in categorical_features]
    
    # Skip if we remove all features of a type
    if len(remaining_numeric) == 0 and len(remaining_categorical) == 0:
        print("  Skipping - would remove all features")
        continue
    
    # Create subset of data
    feature_cols = remaining_features + ["billboard_2012"]
    ml_data_subset = ml_data[feature_cols].dropna()

    X_subset = ml_data_subset.drop(columns=["billboard_2012"])
    y_subset = ml_data_subset["billboard_2012"]

    # Create new preprocessor for remaining features
    transformers = []
    if len(remaining_numeric) > 0:
        transformers.append(('num', StandardScaler(), remaining_numeric))
    if len(remaining_categorical) > 0:
        transformers.append(('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), remaining_categorical))
    
    test_preprocessor = ColumnTransformer(transformers=transformers)
    
    # Create pipeline
    test_pipeline = make_pipeline(
        test_preprocessor,
        LogisticRegression(max_iter=5000)
    )
    
    # Run cross-validation
    X_train_subset, _, y_train_subset, _ = train_test_split(
        X_subset, y_subset, test_size=0.2, random_state=42, stratify=y_subset
    )
    
    cv_scores = cross_val_score(test_pipeline, X_train_subset, y_train_subset, cv=cv, scoring="roc_auc")
    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    
    improvement = mean_score - baseline_score
    feature_removal_results[feature_to_remove] = {
        'mean_score': mean_score,
        'std_score': std_score,
        'improvement': improvement
    }
    
    print(f"  Mean ROC-AUC: {mean_score:.3f} ± {std_score:.3f}")
    print(f"  Change: {improvement:+.3f}")
    print()

# Summary of results
print("=" * 60)
print("SUMMARY - Feature Removal Results:")
print("=" * 60)

# Sort by improvement
sorted_results = sorted(feature_removal_results.items(), key=lambda x: x[1]['improvement'], reverse=True)

for feature, results in sorted_results:
    status = "🟢 IMPROVE" if results['improvement'] > 0 else "🔴 WORSEN"
    print(f"{status} Remove '{feature}': {results['mean_score']:.3f} ({results['improvement']:+.3f})")

# Identify features to remove
features_to_remove = [feature for feature, results in feature_removal_results.items() 
                     if results['improvement'] > 0.01]  # Improvement > 1%

Testing feature removal with cross-validation:
Baseline (all features): 0.513
--------------------------------------------------
Testing removal of: release_year
  Mean ROC-AUC: 0.524 ± 0.111
  Change: +0.011

Testing removal of: type
  Mean ROC-AUC: 0.504 ± 0.115
  Change: -0.009

Testing removal of: genre
  Mean ROC-AUC: 0.563 ± 0.125
  Change: +0.050

Testing removal of: album_name_word_count
  Mean ROC-AUC: 0.550 ± 0.112
  Change: +0.037

Testing removal of: artist_gender
  Mean ROC-AUC: 0.547 ± 0.115
  Change: +0.034

Testing removal of: debut_album_release_year
  Mean ROC-AUC: 0.527 ± 0.111
  Change: +0.014

Testing removal of: years_between
  Mean ROC-AUC: 0.527 ± 0.110
  Change: +0.014

Testing removal of: artist_member_count
  Mean ROC-AUC: 0.541 ± 0.113
  Change: +0.028

SUMMARY - Feature Removal Results:
🟢 IMPROVE Remove 'genre': 0.563 (+0.050)
🟢 IMPROVE Remove 'album_name_word_count': 0.550 (+0.037)
🟢 IMPROVE Remove 'artist_gender': 0.547 (+0.034)
🟢 IMPROVE Remove 'artist_m

Based on the results I will remove some features. But must all features from the subgroup go? Let's see.

In [30]:
# Fit the pipeline to see feature importance
pipeline.fit(X_train, y_train)

# Get feature names after preprocessing
feature_names = pipeline.named_steps['columntransformer'].get_feature_names_out()

# Get coefficients (feature importance for logistic regression)
coefficients = pipeline.named_steps['logisticregression'].coef_[0]

# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': np.abs(coefficients)
}).sort_values('importance', ascending=False)

print("Feature Importance (absolute coefficients):")
print(feature_importance)

Feature Importance (absolute coefficients):
                                           feature  importance
7                                 cat__type_Studio    0.911969
9                            cat__genre_Funk/Disco    0.821110
6                                   cat__type_Live    0.798671
17                      cat__genre_Soul/Gospel/R&B    0.630385
14    cat__genre_Punk/Post-Punk/New Wave/Power Pop    0.463549
13                                cat__genre_Other    0.371436
10                      cat__genre_Hard Rock/Metal    0.354022
8   cat__genre_Country/Folk/Country Rock/Folk Rock    0.347948
19                  cat__artist_gender_Male/Female    0.315940
0                                num__release_year    0.284214
11                          cat__genre_Hip-Hop/Rap    0.210763
16     cat__genre_Singer-Songwriter/Heartland Rock    0.207036
3                               num__years_between    0.163764
2                    num__debut_album_release_year    0.153259
5          

In [31]:
ml_data_engineered = ml_data.copy()
ml_data_engineered["is_funk/disco"] = (ml_data["genre"] == "Funk/Disco").astype(int)
ml_data_engineered["is_soul/gospel"] = (ml_data["genre"] == "Soul/Gospel/R&B").astype(int)
ml_data_engineered = ml_data_engineered.drop(columns=["genre", "album_name_word_count", "years_between", "artist_member_count", "artist_gender", "debut_album_release_year", "release_year"])

In [32]:
categorical_features = ["type", "is_funk/disco", "is_soul/gospel"]
X = ml_data_engineered.drop(columns=["billboard_2012"])
y = ml_data_engineered["billboard_2012"]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=42
) # n_repeats=20, because I want as robust results as possible
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), categorical_features)
    ]
)

# Pipeline
pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=5000)
)

scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc")
print(scores)

[0.65202703 0.62837838 0.6130137  0.66210046 0.43835616 0.5
 0.60675676 0.65753425 0.67009132 0.71232877 0.64752252 0.6027027
 0.57305936 0.68949772 0.49315068 0.73761261 0.58648649 0.52739726
 0.56621005 0.71917808 0.62837838 0.59864865 0.57876712 0.65182648
 0.67351598 0.58108108 0.50675676 0.66552511 0.54452055 0.55479452
 0.56193694 0.4027027  0.65525114 0.55593607 0.67922374 0.65990991
 0.48648649 0.66666667 0.6609589  0.63926941 0.66216216 0.56351351
 0.47260274 0.67694064 0.7260274  0.49324324 0.87297297 0.49885845
 0.47945205 0.59589041 0.74774775 0.55405405 0.69292237 0.56278539
 0.58561644 0.56981982 0.54054054 0.67123288 0.57534247 0.62785388
 0.62725225 0.58783784 0.59246575 0.6826484  0.6369863  0.60698198
 0.57162162 0.53310502 0.60616438 0.66210046 0.49099099 0.50675676
 0.57077626 0.75342466 0.5673516  0.52702703 0.5        0.62328767
 0.70319635 0.67579909 0.5990991  0.65       0.70091324 0.57305936
 0.60958904 0.5990991  0.60675676 0.44063927 0.64383562 0.57990868
 0.

In [33]:
# Fit the pipeline before making predictions
pipeline.fit(X_train, y_train)

# Correct way to get ROC-AUC
y_pred_proba = pipeline.predict_proba(X_holdout)[:, 1]  # Probabilities for class 1
holdout_auc = roc_auc_score(y_holdout, y_pred_proba)
print(f"Holdout ROC-AUC: {holdout_auc:.3f}")

# For classification report
y_pred = pipeline.predict(X_holdout)  # Hard predictions
print(classification_report(y_holdout, y_pred))

Holdout ROC-AUC: 0.504
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.93      1.00      0.96        93

    accuracy                           0.93       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.86      0.93      0.90       100



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

In [34]:
# Your current baseline
baseline_score = 0.504

# Test removing each feature with full CV
current_features = ["type", "is_funk/disco", "is_soul/gospel"]

print("Testing feature removal with cross-validation:")
print(f"Baseline (all features): {baseline_score:.3f}")
print("-" * 50)

feature_removal_results = {}

for feature_to_remove in current_features:
    print(f"Testing removal of: {feature_to_remove}")
    
    # Create feature lists without this feature
    remaining_features = [f for f in current_features if f != feature_to_remove]
    remaining_numeric = [f for f in remaining_features if f in numeric_features]
    remaining_categorical = [f for f in remaining_features if f in categorical_features]
    
    # Skip if we remove all features of a type
    if len(remaining_numeric) == 0 and len(remaining_categorical) == 0:
        print("  Skipping - would remove all features")
        continue
    
    # Create subset of data
    feature_cols = remaining_features + ["billboard_2012"]
    ml_data_subset = ml_data_engineered[feature_cols].dropna()

    X_subset = ml_data_subset.drop(columns=["billboard_2012"])
    y_subset = ml_data_subset["billboard_2012"]

    # Create new preprocessor for remaining features
    transformers = []
    if len(remaining_numeric) > 0:
        transformers.append(('num', StandardScaler(), remaining_numeric))
    if len(remaining_categorical) > 0:
        transformers.append(('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), remaining_categorical))
    
    test_preprocessor = ColumnTransformer(transformers=transformers)
    
    # Create pipeline
    test_pipeline = make_pipeline(
        test_preprocessor,
        LogisticRegression(max_iter=5000)
    )
    
    # Run cross-validation
    X_train_subset, _, y_train_subset, _ = train_test_split(
        X_subset, y_subset, test_size=0.2, random_state=42, stratify=y_subset
    )
    
    cv_scores = cross_val_score(test_pipeline, X_train_subset, y_train_subset, cv=cv, scoring="roc_auc")
    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    
    improvement = mean_score - baseline_score
    feature_removal_results[feature_to_remove] = {
        'mean_score': mean_score,
        'std_score': std_score,
        'improvement': improvement
    }
    
    print(f"  Mean ROC-AUC: {mean_score:.3f} ± {std_score:.3f}")
    print(f"  Change: {improvement:+.3f}")
    print()

# Summary of results
print("=" * 60)
print("SUMMARY - Feature Removal Results:")
print("=" * 60)

# Sort by improvement
sorted_results = sorted(feature_removal_results.items(), key=lambda x: x[1]['improvement'], reverse=True)

for feature, results in sorted_results:
    status = "🟢 IMPROVE" if results['improvement'] > 0 else "🔴 WORSEN"
    print(f"{status} Remove '{feature}': {results['mean_score']:.3f} ({results['improvement']:+.3f})")

# Identify features to remove
features_to_remove = [feature for feature, results in feature_removal_results.items() 
                     if results['improvement'] > 0.01]  # Improvement > 1%

Testing feature removal with cross-validation:
Baseline (all features): 0.504
--------------------------------------------------
Testing removal of: type
  Mean ROC-AUC: 0.541 ± 0.054
  Change: +0.037

Testing removal of: is_funk/disco
  Mean ROC-AUC: 0.589 ± 0.088
  Change: +0.085

Testing removal of: is_soul/gospel
  Mean ROC-AUC: 0.598 ± 0.095
  Change: +0.094

SUMMARY - Feature Removal Results:
🟢 IMPROVE Remove 'is_soul/gospel': 0.598 (+0.094)
🟢 IMPROVE Remove 'is_funk/disco': 0.589 (+0.085)
🟢 IMPROVE Remove 'type': 0.541 (+0.037)


In [35]:
# Fit the pipeline to see feature importance
pipeline.fit(X_train, y_train)

# Get feature names after preprocessing
feature_names = pipeline.named_steps['columntransformer'].get_feature_names_out()

# Get coefficients (feature importance for logistic regression)
coefficients = pipeline.named_steps['logisticregression'].coef_[0]

# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': np.abs(coefficients)
}).sort_values('importance', ascending=False)

print("Feature Importance (absolute coefficients):")
print(feature_importance)

Feature Importance (absolute coefficients):
                   feature  importance
2         cat__type_Studio    1.061205
1           cat__type_Live    0.926322
3     cat__is_funk/disco_1    0.816468
4    cat__is_soul/gospel_1    0.541109
0  cat__type_Greatest Hits    0.173603


In [36]:
ml_data_engineered_2 = ml_data_engineered.copy()
ml_data_engineered_2["is_live"] = (ml_data["type"] == "Live").astype(int)
ml_data_engineered_2["is_studio"] = (ml_data["type"] == "Studio").astype(int)


In [37]:
categorical_features = ["is_funk/disco", "is_studio", "is_live"]
X = ml_data_engineered_2.drop(columns=["billboard_2012"])
y = ml_data_engineered_2["billboard_2012"]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=42
) # n_repeats=20, because I want as robust results as possible
preprocessor = ColumnTransformer(
    transformers=[
        # Only add StandardScaler if numeric_features is not empty

        ('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), categorical_features)
    ]
)

# Pipeline
pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=5000)
)

scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc")
print(scores)

[0.61711712 0.78648649 0.58105023 0.63242009 0.4109589  0.44594595
 0.58243243 0.63926941 0.63013699 0.71917808 0.62387387 0.5472973
 0.69863014 0.67351598 0.46575342 0.70608108 0.54864865 0.48630137
 0.55022831 0.70890411 0.61036036 0.57702703 0.55022831 0.62100457
 0.63242009 0.55067568 0.47972973 0.60958904 0.81506849 0.51826484
 0.5472973  0.66351351 0.63242009 0.51940639 0.65068493 0.62837838
 0.46621622 0.64155251 0.61415525 0.62785388 0.64189189 0.55675676
 0.43835616 0.6369863  0.71575342 0.45945946 0.86891892 0.70890411
 0.45205479 0.55022831 0.73648649 0.49324324 0.67123288 0.5239726
 0.55593607 0.54391892 0.48648649 0.80821918 0.53652968 0.61415525
 0.61486486 0.56621622 0.55593607 0.64840183 0.60502283 0.54504505
 0.55675676 0.53082192 0.72260274 0.64611872 0.71621622 0.44594595
 0.53538813 0.74315068 0.53767123 0.45945946 0.46621622 0.61187215
 0.80593607 0.64155251 0.56418919 0.64459459 0.65981735 0.52968037
 0.60730594 0.55968468 0.5527027  0.70547945 0.62785388 0.561643

In [38]:
pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_holdout)[:, 1]  # Probabilities for class 1
holdout_auc = roc_auc_score(y_holdout, y_pred_proba)
print(f"Holdout ROC-AUC: {holdout_auc:.3f}")

# For classification report
y_pred = pipeline.predict(X_holdout)  # Hard predictions
print(classification_report(y_holdout, y_pred))

Holdout ROC-AUC: 0.575
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.93      1.00      0.96        93

    accuracy                           0.93       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.86      0.93      0.90       100



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

In [40]:

# Your current baseline
baseline_score = 0.575

# Test removing each feature with full CV
current_features = ["is_funk/disco", "is_studio", "is_live"]

print("Testing feature removal with cross-validation:")
print(f"Baseline (all features): {baseline_score:.3f}")
print("-" * 50)

feature_removal_results = {}

for feature_to_remove in current_features:
    print(f"Testing removal of: {feature_to_remove}")
    
    # Create feature lists without this feature
    remaining_features = [f for f in current_features if f != feature_to_remove]
    remaining_numeric = [f for f in remaining_features if f in numeric_features]
    remaining_categorical = [f for f in remaining_features if f in categorical_features]
    
    # Skip if we remove all features of a type
    if len(remaining_numeric) == 0 and len(remaining_categorical) == 0:
        print("  Skipping - would remove all features")
        continue
    
    # Create subset of data
    feature_cols = remaining_features + ["bilboard_2012"]
    ml_data_subset = ml_data_engineered_2[feature_cols].dropna()

    X_subset = ml_data_subset.drop(columns=["bilboard_2012"])
    y_subset = ml_data_subset["bilboard_2012"]

    # Create new preprocessor for remaining features
    transformers = []
    if len(remaining_numeric) > 0:
        transformers.append(('num', StandardScaler(), remaining_numeric))
    if len(remaining_categorical) > 0:
        transformers.append(('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), remaining_categorical))
    
    test_preprocessor = ColumnTransformer(transformers=transformers)
    
    # Create pipeline
    test_pipeline = make_pipeline(
        test_preprocessor,
        LogisticRegression(max_iter=5000)
    )
    
    # Run cross-validation
    X_train_subset, _, y_train_subset, _ = train_test_split(
        X_subset, y_subset, test_size=0.2, random_state=42, stratify=y_subset
    )
    
    cv_scores = cross_val_score(test_pipeline, X_train_subset, y_train_subset, cv=cv, scoring="roc_auc")
    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    
    improvement = mean_score - baseline_score
    feature_removal_results[feature_to_remove] = {
        'mean_score': mean_score,
        'std_score': std_score,
        'improvement': improvement
    }
    
    print(f"  Mean ROC-AUC: {mean_score:.3f} ± {std_score:.3f}")
    print(f"  Change: {improvement:+.3f}")
    print()

# Summary of results
print("=" * 60)
print("SUMMARY - Feature Removal Results:")
print("=" * 60)

# Sort by improvement
sorted_results = sorted(feature_removal_results.items(), key=lambda x: x[1]['improvement'], reverse=True)

for feature, results in sorted_results:
    status = "🟢 IMPROVE" if results['improvement'] > 0 else "🔴 WORSEN"
    print(f"{status} Remove '{feature}': {results['mean_score']:.3f} ({results['improvement']:+.3f})")

# Identify features to remove
features_to_remove = [feature for feature, results in feature_removal_results.items() 
                     if results['improvement'] > 0.01]  # Improvement > 1%

Testing feature removal with cross-validation:
Baseline (all features): 0.575
--------------------------------------------------
Testing removal of: is_funk/disco
  Mean ROC-AUC: 0.582 ± 0.101
  Change: +0.007

Testing removal of: is_studio
  Mean ROC-AUC: 0.541 ± 0.015
  Change: -0.034

Testing removal of: is_live
  Mean ROC-AUC: 0.591 ± 0.095
  Change: +0.016

SUMMARY - Feature Removal Results:
🟢 IMPROVE Remove 'is_live': 0.591 (+0.016)
🟢 IMPROVE Remove 'is_funk/disco': 0.582 (+0.007)
🔴 WORSEN Remove 'is_studio': 0.541 (-0.034)


In [41]:
# Fit the pipeline to see feature importance
pipeline.fit(X_train, y_train)

# Get feature names after preprocessing
feature_names = pipeline.named_steps['columntransformer'].get_feature_names_out()

# Get coefficients (feature importance for logistic regression)
coefficients = pipeline.named_steps['logisticregression'].coef_[0]

# Create feature importance dataframe
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': np.abs(coefficients)
}).sort_values('importance', ascending=False)

print("Feature Importance (absolute coefficients):")
print(feature_importance)

Feature Importance (absolute coefficients):
                feature  importance
1      cat__is_studio_1    1.024962
2        cat__is_live_1    0.928464
0  cat__is_funk/disco_1    0.775305


In [42]:
ml_data_engineered_3 = ml_data.copy()

ml_data_engineered_3["is_live"] = (ml_data["type"] == "Live").astype(int)
ml_data_engineered_3["is_studio"] = (ml_data["type"] == "Studio").astype(int)
ml_data_engineered_3 = ml_data_engineered_3.drop(columns=["genre", "album_name_word_count", "years_between", "artist_member_count", "artist_gender"])

In [43]:

categorical_features = ["is_studio", "is_live"]
X = ml_data_engineered_3.drop(columns=["bilboard_2012"])
y = ml_data_engineered_3["bilboard_2012"]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=42
) # n_repeats=20, because I want as robust results as possible
preprocessor = ColumnTransformer(
    transformers=[
        # Only add StandardScaler if numeric_features is not empty
        ('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), categorical_features)
    ]
)

# Pipeline
pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=5000)
)

scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc")
print(scores)

[0.6036036  0.77162162 0.55251142 0.63013699 0.39041096 0.42567568
 0.56081081 0.61643836 0.60502283 0.70890411 0.60585586 0.5472973
 0.68493151 0.65068493 0.42465753 0.68918919 0.51486486 0.45890411
 0.543379   0.70547945 0.59684685 0.55540541 0.53881279 0.59817352
 0.61415525 0.53153153 0.43918919 0.59817352 0.80821918 0.50570776
 0.50788288 0.64324324 0.63013699 0.51141553 0.64383562 0.61036036
 0.43243243 0.6369863  0.59817352 0.61872146 0.61711712 0.54594595
 0.43150685 0.62100457 0.69520548 0.4527027  0.86351351 0.68150685
 0.43150685 0.53767123 0.72297297 0.46621622 0.65753425 0.50570776
 0.54452055 0.53040541 0.43243243 0.80365297 0.51826484 0.60730594
 0.60585586 0.54864865 0.51369863 0.63013699 0.60273973 0.52702703
 0.52432432 0.51826484 0.71232877 0.63926941 0.69932432 0.42567568
 0.50684932 0.72945205 0.52054795 0.43243243 0.4527027  0.59817352
 0.78995434 0.61872146 0.54166667 0.64054054 0.6369863  0.51141553
 0.59589041 0.53040541 0.53513514 0.72260274 0.61415525 0.53196

In [44]:
pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_holdout)[:, 1]  # Probabilities for class 1
holdout_auc = roc_auc_score(y_holdout, y_pred_proba)
print(f"Holdout ROC-AUC: {holdout_auc:.3f}")

# For classification report
y_pred = pipeline.predict(X_holdout)  # Hard predictions
print(classification_report(y_holdout, y_pred))

Holdout ROC-AUC: 0.563
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.93      1.00      0.96        93

    accuracy                           0.93       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.86      0.93      0.90       100



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

In [45]:
# Your current baseline
baseline_score = 0.563

# Test removing each feature with full CV
current_features = ["is_studio", "is_live"]

print("Testing feature removal with cross-validation:")
print(f"Baseline (all features): {baseline_score:.3f}")
print("-" * 50)

feature_removal_results = {}

for feature_to_remove in current_features:
    print(f"Testing removal of: {feature_to_remove}")
    
    # Create feature lists without this feature
    remaining_features = [f for f in current_features if f != feature_to_remove]
    remaining_numeric = [f for f in remaining_features if f in numeric_features]
    remaining_categorical = [f for f in remaining_features if f in categorical_features]
    
    # Skip if we remove all features of a type
    if len(remaining_numeric) == 0 and len(remaining_categorical) == 0:
        print("  Skipping - would remove all features")
        continue
    
    # Create subset of data
    feature_cols = remaining_features + ["bilboard_2012"]
    ml_data_subset = ml_data_engineered_3[feature_cols].dropna()

    X_subset = ml_data_subset.drop(columns=["bilboard_2012"])
    y_subset = ml_data_subset["bilboard_2012"]

    # Create new preprocessor for remaining features
    transformers = []
    if len(remaining_numeric) > 0:
        transformers.append(('num', StandardScaler(), remaining_numeric))
    if len(remaining_categorical) > 0:
        transformers.append(('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), remaining_categorical))
    
    test_preprocessor = ColumnTransformer(transformers=transformers)
    
    # Create pipeline
    test_pipeline = make_pipeline(
        test_preprocessor,
        LogisticRegression(max_iter=5000)
    )
    
    # Run cross-validation
    X_train_subset, _, y_train_subset, _ = train_test_split(
        X_subset, y_subset, test_size=0.2, random_state=42, stratify=y_subset
    )
    
    cv_scores = cross_val_score(test_pipeline, X_train_subset, y_train_subset, cv=cv, scoring="roc_auc")
    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    
    improvement = mean_score - baseline_score
    feature_removal_results[feature_to_remove] = {
        'mean_score': mean_score,
        'std_score': std_score,
        'improvement': improvement
    }
    
    print(f"  Mean ROC-AUC: {mean_score:.3f} ± {std_score:.3f}")
    print(f"  Change: {improvement:+.3f}")
    print()

# Summary of results
print("=" * 60)
print("SUMMARY - Feature Removal Results:")
print("=" * 60)

# Sort by improvement
sorted_results = sorted(feature_removal_results.items(), key=lambda x: x[1]['improvement'], reverse=True)

for feature, results in sorted_results:
    status = "🟢 IMPROVE" if results['improvement'] > 0 else "🔴 WORSEN"
    print(f"{status} Remove '{feature}': {results['mean_score']:.3f} ({results['improvement']:+.3f})")

# Identify features to remove
features_to_remove = [feature for feature, results in feature_removal_results.items() 
                     if results['improvement'] > 0.01]  # Improvement > 1%

Testing feature removal with cross-validation:
Baseline (all features): 0.563
--------------------------------------------------
Testing removal of: is_studio
  Mean ROC-AUC: 0.519 ± 0.010
  Change: -0.044

Testing removal of: is_live
  Mean ROC-AUC: 0.575 ± 0.096
  Change: +0.012

SUMMARY - Feature Removal Results:
🟢 IMPROVE Remove 'is_live': 0.575 (+0.012)
🔴 WORSEN Remove 'is_studio': 0.519 (-0.044)


In [46]:
categorical_features = ["is_studio"]
X = ml_data_engineered_3[["is_studio"]]  # Use double brackets to get a DataFrame
y = ml_data_engineered_3["bilboard_2012"]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=42
) # n_repeats=20, because I want as robust results as possible
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), categorical_features)
    ]
)

# Pipeline
pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=5000)
)

scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc")
print(scores)

[0.5990991  0.7527027  0.54908676 0.61872146 0.39041096 0.42567568
 0.55945946 0.60502283 0.59817352 0.68835616 0.5990991  0.54594595
 0.66780822 0.63926941 0.42465753 0.67567568 0.50540541 0.45890411
 0.54223744 0.69520548 0.58558559 0.5527027  0.53538813 0.5913242
 0.61187215 0.52927928 0.43918919 0.5913242  0.79908676 0.50114155
 0.50225225 0.63243243 0.61872146 0.50799087 0.63242009 0.5990991
 0.43243243 0.62557078 0.5913242  0.61187215 0.61261261 0.54594595
 0.43150685 0.61187215 0.67465753 0.4527027  0.83918919 0.66780822
 0.43150685 0.53538813 0.7027027  0.46621622 0.64726027 0.50114155
 0.54223744 0.52927928 0.43243243 0.78538813 0.51484018 0.59817352
 0.5990991  0.54594595 0.50799087 0.62557078 0.59817352 0.52252252
 0.51891892 0.51484018 0.68150685 0.63926941 0.68918919 0.42567568
 0.50114155 0.72945205 0.51484018 0.43243243 0.4527027  0.5913242
 0.7716895  0.61187215 0.53603604 0.63243243 0.62557078 0.50799087
 0.5913242  0.52927928 0.53243243 0.68150685 0.60502283 0.5285388

In [47]:
pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_holdout)[:, 1]  # Probabilities for class 1
holdout_auc = roc_auc_score(y_holdout, y_pred_proba)
print(f"Holdout ROC-AUC: {holdout_auc:.3f}")

# For classification report
y_pred = pipeline.predict(X_holdout)  # Hard predictions
print(classification_report(y_holdout, y_pred))

Holdout ROC-AUC: 0.568
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.93      1.00      0.96        93

    accuracy                           0.93       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.86      0.93      0.90       100



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Although the ROC-AUC improved slightly, the model performed best at 0.57 with the following features:
- `is_funk/disco`
- `is_studio` 
- `is_live`

## FINAL MODEL *again* 


In [49]:
ml_data_engineered_2 = ml_data_engineered.copy()
ml_data_engineered_2["is_live"] = (ml_data["type"] == "Live").astype(int)
ml_data_engineered_2["is_studio"] = (ml_data["type"] == "Studio").astype(int)

In [50]:
categorical_features = ["is_funk/disco", "is_studio", "is_live"]
X = ml_data_engineered_2.drop(columns=["bilboard_2012"])
y = ml_data_engineered_2["bilboard_2012"]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cv = RepeatedStratifiedKFold(
    n_splits=5,
    n_repeats=20,
    random_state=42
) # n_repeats=20, because I want as robust results as possible
preprocessor = ColumnTransformer(
    transformers=[
        # Only add StandardScaler if numeric_features is not empty

        ('cat', OneHotEncoder(drop='first', handle_unknown='infrequent_if_exist', min_frequency=2), categorical_features)
    ]
)

# Pipeline
pipeline = make_pipeline(
    preprocessor,
    LogisticRegression(max_iter=5000)
)

scores = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring="roc_auc")
print(scores)

[0.61711712 0.78648649 0.58105023 0.63242009 0.4109589  0.44594595
 0.58243243 0.63926941 0.63013699 0.71917808 0.62387387 0.5472973
 0.69863014 0.67351598 0.46575342 0.70608108 0.54864865 0.48630137
 0.55022831 0.70890411 0.61036036 0.57702703 0.55022831 0.62100457
 0.63242009 0.55067568 0.47972973 0.60958904 0.81506849 0.51826484
 0.5472973  0.66351351 0.63242009 0.51940639 0.65068493 0.62837838
 0.46621622 0.64155251 0.61415525 0.62785388 0.64189189 0.55675676
 0.43835616 0.6369863  0.71575342 0.45945946 0.86891892 0.70890411
 0.45205479 0.55022831 0.73648649 0.49324324 0.67123288 0.5239726
 0.55593607 0.54391892 0.48648649 0.80821918 0.53652968 0.61415525
 0.61486486 0.56621622 0.55593607 0.64840183 0.60502283 0.54504505
 0.55675676 0.53082192 0.72260274 0.64611872 0.71621622 0.44594595
 0.53538813 0.74315068 0.53767123 0.45945946 0.46621622 0.61187215
 0.80593607 0.64155251 0.56418919 0.64459459 0.65981735 0.52968037
 0.60730594 0.55968468 0.5527027  0.70547945 0.62785388 0.561643

In [51]:
pipeline.fit(X_train, y_train)

y_pred_proba = pipeline.predict_proba(X_holdout)[:, 1]  # Probabilities for class 1
holdout_auc = roc_auc_score(y_holdout, y_pred_proba)
print(f"Holdout ROC-AUC: {holdout_auc:.3f}")

# For classification report
y_pred = pipeline.predict(X_holdout)  # Hard predictions
print(classification_report(y_holdout, y_pred))

Holdout ROC-AUC: 0.575
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.93      1.00      0.96        93

    accuracy                           0.93       100
   macro avg       0.47      0.50      0.48       100
weighted avg       0.86      0.93      0.90       100



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

# Final model- predicting the 2020 dataset

In [52]:
#Adding bilboard 2020 data to ml_data_engineered_2
ml_data_engineered_2["bilboard_2020"] = rs_data["bilboard_2020"].copy()

In [53]:
y_pred_2020 = pipeline.predict(ml_data_engineered_2[categorical_features])
ml_data_engineered_2["pred_2020"] = y_pred_2020



In [56]:
#Comparing to the actual 2020 billboard presence

print(confusion_matrix(ml_data_engineered_2['bilboard_2020'], ml_data_engineered_2['pred_2020']))
print(classification_report(ml_data_engineered_2['bilboard_2020'], ml_data_engineered_2['pred_2020']))

[[  0 171]
 [  0 325]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       171
           1       0.66      1.00      0.79       325

    accuracy                           0.66       496
   macro avg       0.33      0.50      0.40       496
weighted avg       0.43      0.66      0.52       496



/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nemethtamas/opt/anaconda3/envs/rolling_stones/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavi

Based on the model results, it predicts that all albums will be on the Billboard charts. This suggests we cannot predict future success from 2012 data alone. Note that I used only albums that were on the Billboard charts in 2003. There can be other important factors contributing to an album's success that our model doesn't capture. However, we must acknowledge that the data was not sufficient to predict success accurately. The model was also biased, as many more albums remained on the Billboard charts than dropped off. For a more accurate and comprehensive analysis, we need albums that were **not** on the Billboard charts in 2003 to create a more balanced dataset. 

Also note that the original dataset contained albums that were older than 2003. Since I was curious what makes an album great in the long run, newer albums were not within the scope of this analysis.

---

## 🎯 Key Insights & Business Impact

### **Model Performance Summary**
- **Best ROC-AUC**: 0.57 (meaningful improvement over random chance)
- **Most Important Features**: Album type (Studio vs Live) and genre (Funk/Disco)
- **Prediction Challenge**: Severe class imbalance (most albums remained on charts)

### **Business Insights**

**1. Album Longevity Patterns**
- **92% survival rate** from 2003 to 2012 (9 years)
- **64% survival rate** from 2003 to 2020 (17 years)
- Classic albums show remarkable staying power

**2. Key Success Factors**
- **Album Format**: Studio albums vs Live albums have different survival patterns
- **Genre Influence**: Funk/Disco albums show distinct longevity characteristics
- **Market Stability**: The "classic" canon is relatively stable over time

### **Technical Achievements**

**1. Innovation in Data Science**
- Successfully used **GPT-4 for data augmentation** (60% accuracy)
- Implemented **LLM-as-a-Judge** validation framework
- Combined traditional ML with modern AI tools

**2. Methodological Rigor**
- **Survival Analysis**: Kaplan-Meier estimators for time-to-event modeling
- **Robust Validation**: 20-repeat stratified cross-validation
- **Systematic Feature Engineering**: Data-driven feature selection process

### **Challenges & Limitations**

**1. Data Limitations**
- **Class Imbalance**: Most albums remained on charts (limited negative examples)
- **Temporal Scope**: Only albums from 2003 baseline (missing broader context)
- **Missing Features**: No streaming data, sales figures, or cultural impact metrics

**2. Model Limitations**
- **Overfitting Tendency**: Model predicts most albums will remain successful
- **Limited Predictive Power**: ROC-AUC of 0.57 indicates modest performance
- **Bias**: Dataset skewed toward already successful albums

### **Future Improvements**

**1. Data Enhancement**
- Include albums **not** on 2003 charts for better class balance
- Add external data: streaming numbers, sales figures, cultural impact scores
- Expand temporal range to include more recent albums

**2. Model Improvements**
- Apply **class balancing techniques** (SMOTE, undersampling)
- Try **ensemble methods** (Random Forest, XGBoost)
- Implement **time-series analysis** for temporal patterns

**3. Business Applications**
- **Music Industry**: Predict long-term album success
- **Investment**: Identify undervalued music catalogs
- **Curation**: Assist in building "timeless" playlists

---

## 📊 Project Summary

This analysis demonstrates the intersection of **traditional statistics** (survival analysis), **modern machine learning** (sklearn pipelines), and **cutting-edge AI** (LLM data augmentation). While the predictive model shows modest performance, the methodological approach and innovative techniques showcase advanced data science capabilities suitable for real-world applications.

**Key Takeaway**: Even with limited features, we can identify meaningful patterns in cultural longevity, though additional data and advanced techniques would be needed for production-level prediction accuracy.